# Use Numba and cython speed up Python

Numba provides a "jit" or a "just in time" compiler. Recall that C is a compiled langauge, its source code is optimized and converted to CPU instructions. Python is "interpreted," its code is NOT optimized ahead of time and is interpreted a line at a time. This means that a Python interpreter does not have enough information for optimization.

Just in time (JIT) compiler are used in many languages, including Java, C# and even Python is getting a JIT compiler in version 3.13. Numba is also a JIT addon that we can use with any version.

In [ ]:
#%pip install numba

In [ ]:
import math
import numpy as np
import pandas as pd
from timeit import timeit

### An implementation of K-nearest neighbors classifier
Code from perplexity.io and Claud.ai

#### Python only

In [ ]:
def euclidean_distance(p1, p2):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(p1, p2)))

def knn_python(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        distances = []
        for i, train_point in enumerate(X_train):
            dist = euclidean_distance(test_point, train_point)
            distances.append((dist, y_train[i]))
        distances.sort(key=lambda x: x[0])

        nearest_labels = [label for (_, label) in distances[:k]]

        # Majority vote, manual count
        best_label = None
        best_count = 0
        for label in set(nearest_labels):
            count = nearest_labels.count(label)
            if count > best_count:
                best_count = count
                best_label = label
                
        predictions.append(best_label)
    return predictions


#### Numpy
Numpy could be much faster if it could use specialized functions in its library

In [ ]:
def knn_numpy(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        # Vectorized distance calculation: array subtraction + sum of squares + sqrt
        diff = X_train - test_point  # shape (n_train, n_features)
        dists = np.sqrt(np.sum(diff ** 2, axis=1))  # shape (n_train,)

        # Convert to list of (distance, label) tuples
        distances = list(zip(dists.tolist(), y_train.tolist()))

        # Sort by distance
        distances.sort(key=lambda x: x[0])

        nearest_labels = [label for (_, label) in distances[:k]]

        # Majority vote, manual count
        best_label = None
        best_count = 0
        for label in set(nearest_labels):
            count = nearest_labels.count(label)
            if count > best_count:
                best_count = count
                best_label = label
        
        predictions.append(best_label)
    return predictions


#### Measure

In [ ]:
# Some made up numbers
X_train = [[0.93588381, 0.64083873, 0.43191558],
           [0.42222037, 0.11649837, 0.06062254],
           [0.55167179, 0.92109796, 0.60461583]]
y_train = [1, 2, 3]
X_test  = [[0.35138874453456725, 0.519960657417942, 0.8637018994564011],
             [0.6558387008127351, 0.5067038567727152, 0.17407496226564068],
             [0.34327858991593596, 0.6883918432579191, 0.9240662106562946]]

X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_test_np  = np.array(y_train)

%timeit knn_python(X_train, y_train, X_test, k=2)
%timeit knn_numpy(X_train_np, y_train_np, X_test_np, k=2)

Shouldn't numpy be MUCH faster than basic Python??

#### Scale matters!

In [ ]:
%%time
times = []

COLUMNS = 3

for i in [3, 5, 10, 100, 500, 1_000]:
    X_train_np = np.random.rand(i, COLUMNS)
    y_train_np = np.random.randint(0, 5, size=i)  # Labels from 0 to 4
    X_test_np  = np.random.rand(i, COLUMNS)

    X_train = X_train_np.tolist()
    y_train = y_train_np.tolist()
    X_test  = X_test_np.tolist()

    python_time = timeit(lambda: knn_python(X_train, y_train, X_test, k=2), number=10)
    numpy_time  = timeit(lambda: knn_numpy(X_train_np, y_train_np, X_test_np, k=2), number=10)

    times.append((i, python_time, numpy_time))

# Pretty-print
df = pd.DataFrame(times, columns=["n_samples", "python_time", "numpy_time"])
df

In [ ]:
df.plot.line(x='n_samples')

#### Let's compare numpy with numba

In [ ]:
import numba

In [ ]:
import math
import numpy as np
import numba

@numba.njit
def euclidean_distance_numba(p1, p2):
    total = 0.0
    for i in range(len(p1)):
        diff = p1[i] - p2[i]
        total += diff * diff
    return math.sqrt(total)

@numba.njit
def knn_numba(X_train, y_train, X_test, k):
    n_test = X_test.shape[0]
    predictions = np.empty(n_test, dtype=y_train.dtype)

    for idx in range(n_test):
        test_point = X_test[idx]
        n_train = X_train.shape[0]

        distances = np.empty(n_train)
        for i in range(n_train):
            distances[i] = euclidean_distance_numba(test_point, X_train[i])

        # Simple sort using argsort
        sorted_indices = np.argsort(distances)

        # Majority vote for top k
        label_counts = dict()
        for j in range(k):
            label = y_train[sorted_indices[j]]
            if label in label_counts:
                label_counts[label] += 1
            else:
                label_counts[label] = 1

        best_label = -1
        best_count = -1
        for label, count in label_counts.items():
            if count > best_count:
                best_label = label
                best_count = count

        predictions[idx] = best_label

    return predictions

In [ ]:
%%time
times = []

COLUMNS = 3

for i in [3, 5, 10, 100, 500, 1_000]:
    X_train_np = np.random.rand(i, COLUMNS)
    y_train_np = np.random.randint(0, 5, size=i)  # Labels from 0 to 4
    X_test_np  = np.random.rand(i, COLUMNS)

    X_train = X_train_np.tolist()
    y_train = y_train_np.tolist()
    X_test  = X_test_np.tolist()

    numba_time  = timeit(lambda: knn_numba(X_train_np, y_train_np, X_test_np, k=2), number=10)

    times.append((i, numba_time))

# Pretty-print
df_numba = pd.DataFrame(times, columns=["n_samples", "numba_time"])

In [ ]:
df = pd.concat([df, df_numba.numba_time], axis=1)
df

In [ ]:
df.plot.line(x='n_samples')

#### Warning, `numba` can be very finicky

For example, numba should not be used with standard Python lists, it works _much_ better with numpy arrays. It does not work with the `yield` keyword. Even if you don't use it explicitely, it is used in some comprehensions.

### What if we pre-compile Python, similar to C? `cython` does exactly that

The scikit-learn library uses cython, as do some other Python libraries in the scipy universe.  cython requires well typed variables via the `cdef` keyword. These types give compilers much more information and helps them optimize code.

In [ ]:
#!pip install Cython # make sure it is installed in the correct enviornment

In [ ]:
%load_ext cython

In [ ]:
%%cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt

ctypedef np.float64_t DTYPE_t
ctypedef np.int64_t ITYPE_t

def knn_cython(np.ndarray[DTYPE_t, ndim=2] X_train,
               np.ndarray[ITYPE_t, ndim=1] y_train,
               np.ndarray[DTYPE_t, ndim=2] X_test,
               int k):

    cdef int n_test = X_test.shape[0]
    cdef int n_train = X_train.shape[0]
    cdef int n_features = X_train.shape[1]
    cdef np.ndarray[ITYPE_t, ndim=1] predictions = np.empty(n_test, dtype=np.int64)

    cdef int i, j, idx
    cdef double dist
    cdef double diff
    cdef int best_label
    cdef int best_count
    cdef int label

    cdef np.ndarray[double, ndim=1] distances = np.empty(n_train, dtype=np.float64)
    cdef dict label_counts

    for idx in range(n_test):
        # Compute distances
        for i in range(n_train):
            dist = 0.0
            for j in range(n_features):
                diff = X_train[i, j] - X_test[idx, j]
                dist += diff * diff
            distances[i] = sqrt(dist)

        # Sort distances, get sorted indices
        sorted_indices = distances.argsort()

        # Majority vote
        label_counts = {}
        best_label = -1
        best_count = -1
        for i in range(k):
            label = y_train[sorted_indices[i]]
            if label in label_counts:
                label_counts[label] += 1
            else:
                label_counts[label] = 1

            if label_counts[label] > best_count:
                best_label = label
                best_count = label_counts[label]

        predictions[idx] = best_label

    return predictions

In [ ]:
%%time
times = []

COLUMNS = 3

for i in [3, 5, 10, 100, 500, 1_000]:
    X_train_np = np.random.rand(i, COLUMNS)
    y_train_np = np.random.randint(0, 5, size=i)  # Labels from 0 to 4
    X_test_np  = np.random.rand(i, COLUMNS)

    X_train = X_train_np.tolist()
    y_train = y_train_np.tolist()
    X_test  = X_test_np.tolist()

    cython_time  = timeit(lambda: knn_cython(X_train_np, y_train_np, X_test_np, k=2), number=10)

    times.append((i, cython_time))

# Pretty-print
df_cython = pd.DataFrame(times, columns=["n_samples", "cython_time"])

In [ ]:
df = pd.concat([df, df_cython.cython_time], axis=1)
df

In [ ]:
df.plot.line(x='n_samples')

### Modern Python already has optional types, what if we use those to help compilers?

In [ ]:
#!pip install 'mypy[mypyc]'

In [ ]:
%%writefile mypyc_knn.py
import math
from typing import List
import numpy as np
from numpy.typing import NDArray


def euclidean_distance_mypyc(
        p1: NDArray[np.float64]
        , p2: NDArray[np.float64]) -> float:
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(p1, p2)))


def knn_python_mypyc(
    X_train: NDArray[np.float64],        # shape (n_samples, n_features)
    y_train: NDArray[np.int64],          # shape (n_samples,)
    X_test: NDArray[np.float64],         # shape (n_test, n_features)
    k: int
) -> List[int]:                          # returns list of predicted labels (ints)
    predictions: List[int] = []
    for test_point in X_test:
        distances: List[tuple[float, int]] = []
        for i, train_point in enumerate(X_train):
            dist = euclidean_distance_mypyc(test_point, train_point)
            distances.append((dist, int(y_train[i])))  # cast in case numpy type
        distances.sort(key=lambda x: x[0])

        nearest_labels: List[int] = [label for (_, label) in distances[:k]]

        # Majority vote, manual count
        best_label: int | None = None
        best_count = 0
        for label in set(nearest_labels):
            count = nearest_labels.count(label)
            if count > best_count:
                best_count = count
                best_label = label

        # best_label can’t be None here if k > 0
        predictions.append(best_label if best_label is not None else -1)
    return predictions


In [ ]:
# !python -m mypyc mypyc_knn.py

# Compile using the mleng environment explicitly (unfortunately)
!conda run -n mleng python -m mypyc mypyc_knn.py

In [ ]:
# Remove the src file to force Python to load the .so file
!rm mypyc_knn.py

In [ ]:
import mypyc_knn
print(mypyc_knn.__file__)

In [ ]:
%%time
times = []

COLUMNS = 3

for i in [3, 5, 10, 100, 500, 1_000]:
    X_train_np = np.random.rand(i, COLUMNS)
    y_train_np = np.random.randint(0, 5, size=i)  # Labels from 0 to 4
    X_test_np  = np.random.rand(i, COLUMNS)

    X_train = X_train_np.tolist()
    y_train = y_train_np.tolist()
    X_test  = X_test_np.tolist()

    mypyc_time  = timeit(lambda: mypyc_knn.knn_python_mypyc(X_train_np, y_train_np, X_test_np, k=2), number=10)

    times.append((i, mypyc_time))

# Pretty-print
df_mypyc = pd.DataFrame(times, columns=["n_samples", "mypyc_time"])

In [ ]:
df_mypyc

In [ ]:
df = pd.concat([df, df_mypyc.mypyc_time], axis=1)
df

In [ ]:
df.plot.line(x='n_samples')

As you can see, `mypyc` is not able to optimize code as much as `cython`. Data science related code may not be mypyc's best use case.

### `Mojo` is a new Python derivative, built by a VERY strong tech team
We will leave experimenting with it for later time:

https://www.modular.com/mojo